In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

  Attempting uninstall: aiosqlite
    Found existing installation: aiosqlite 0.18.0
    Uninstalling aiosqlite-0.18.0:
      Successfully uninstalled aiosqlite-0.18.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("langchain api: ")
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("anthropic api: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("tavily api: ")

langchain api: ········
anthropic api: ········
tavily api: ········


In [12]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Hyderabad")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Hyderabad', 'region': 'Andhra Pradesh', 'country': 'India', 'lat': 17.38, 'lon': 78.47, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1724659185, 'localtime': '2024-08-26 13:29'}, 'current': {'last_updated_epoch': 1724658300, 'last_updated': '2024-08-26 13:15', 'temp_c': 29.1, 'temp_f': 84.4, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 16.1, 'wind_kph': 25.9, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1008.0, 'pressure_in': 29.77, 'precip_mm': 0.03, 'precip_in': 0.0, 'humidity': 62, 'cloud': 50, 'feelslike_c': 32.2, 'feelslike_f': 90.0, 'windchill_c': 28.2, 'windchill_f': 82.8, 'heatindex_c': 30.6, 'heatindex_f': 87.1, 'dewpoint_c': 21.0, 'dewpoint_f': 69.8, 'vis_km': 5.0, 'vis_miles': 3.0, 'uv': 6.0, 'gust_mph': 21.1, 'gust_kph': 33.9}}"}, {'url': 'https://world-weather.info/forecast/india/hyderabad/august-2024/', 'conten

In [13]:
pip install -qU langchain-anthropic

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [15]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi, what is your view on india and ai!")])
response.content

"As an AI language model, I don't have personal views or opinions. However, I can provide you with some objective information about India and AI:\n\nIndia has been making significant strides in the field of artificial intelligence (AI) in recent years. Here are some key points:\n\n1. Government initiatives: The Indian government has launched several initiatives to promote AI research and development, such as the National AI Strategy and the establishment of AI-focused institutions.\n\n2. Growing tech industry: India's large and growing IT sector provides a strong foundation for AI development and implementation.\n\n3. Talent pool: India produces a large number of engineering and computer science graduates, contributing to a skilled workforce in AI and related fields.\n\n4. AI adoption: Various sectors in India, including healthcare, finance, and agriculture, are increasingly adopting AI solutions to improve efficiency and services.\n\n5. Challenges: India faces challenges in AI develop

In [16]:
model_with_tools = model.bind_tools(tools)

In [17]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! Welcome! How can I assist you today? I'm here to help with any questions or tasks you might have. Please feel free to ask about anything you'd like to know or discuss.
ToolCalls: []


In [11]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I apologize, but I don't have a specific tool available to check the current weather in San Francisco. However, I can search for the latest weather information using the search function available to me. Would you like me to do that?
ToolCalls: []


In [19]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [20]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='dd9e2cab-2408-4316-a3f8-c13eee5fce89'),
 AIMessage(content="Hello! It's nice to meet you. How can I assist you today? I'm here to help with any questions or tasks you might have. Is there anything specific you'd like to know or discuss?", response_metadata={'id': 'msg_013yNTso6qn4GYfAXv4WJaz6', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 399, 'output_tokens': 45}}, id='run-1b0d7760-084d-44e2-86ec-5dd1474007fb-0', usage_metadata={'input_tokens': 399, 'output_tokens': 45, 'total_tokens': 444})]

In [21]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='d42499e0-95f6-4eb8-be9e-5050fc7f87d0'),
 AIMessage(content=[{'text': "I apologize, but I don't have access to real-time weather information or a specific weather tool. However, I can use a search engine to find the most up-to-date weather information for San Francisco. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01FY2qyiCKCJRuZb8JVwDXML', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01JwyFuAQFbzGjzsSCB8CreC', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 404, 'output_tokens': 115}}, id='run-36b705b3-3e89-4802-bb81-2b7cb76341d8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01FY2qyiCKCJRuZb8JVwDXML', 'type': 'tool_call'}], usage_metadata={'input_tokens': 404, 'output_tokens': 115

In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I apologize, but I don't have a specific tool available to check the current weather in San Francisco. However, I can use the search function to find recent weather information for you. Would you like me to do that?", response_metadata={'id': 'msg_0174dCb6TLjS4HpCUwXW8D4A', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 404, 'output_tokens': 49}}, id='run-69b037a2-8d93-4e1e-990b-b5751d3c9e07-0', usage_metadata={'input_tokens': 404, 'output_tokens': 49, 'total_tokens': 453})]}}
----


In [23]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/Users/madhushantan/anaconda3/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


[{'text': 'I', 'type': 'text', 'index': 0}]|[{'text': ' apolog', 'type': 'text', 'index': 0}]|[{'text': 'ize, but I don', 'type': 'text', 'index': 0}]|[{'text': "'t have a", 'type': 'text', 'index': 0}]|[{'text': ' specific', 'type': 'text', 'index': 0}]|[{'text': ' weather tool', 'type': 'text', 'index': 0}]|[{'text': ' available', 'type': 'text', 'index': 0}]|[{'text': ' to provide', 'type': 'text', 'index': 0}]|[{'text': ' real', 'type': 'text', 'index': 0}]|[{'text': '-time weather information for', 'type': 'text', 'index': 0}]|[{'text': ' San Francisco. However', 'type': 'text', 'index': 0}]|[{'text': ', I', 'type': 'text', 'index': 0}]|[{'text': ' can use', 'type': 'text', 'index': 0}]|[{'text': ' the search function', 'type': 'text', 'index': 0}]|[{'text': ' to fin', 'type': 'text', 'index': 0}]|[{'text': 'd the', 'type': 'text', 'index': 0}]|[{'text': ' most', 'type': 'text', 'index': 0}]|[{'text': ' up-to-date', 'type': 'text', 'index': 0}]|[{'text': ' weather information', 't

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [25]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [26]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today? Is there any particular information you're looking for or any questions you have?", response_metadata={'id': 'msg_019CgQTsuJkEj8C3gYHqaxBw', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 401, 'output_tokens': 35}}, id='run-509348c2-8faf-452a-b125-60d423e62550-0', usage_metadata={'input_tokens': 401, 'output_tokens': 35, 'total_tokens': 436})]}}
----


In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. You introduced yourself as "Bob" in your first message.', response_metadata={'id': 'msg_01FaxRvUHvnL6VXrBtzy6XPV', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 444, 'output_tokens': 20}}, id='run-6f4db26d-3213-4fa7-a98d-c32f030527fd-0', usage_metadata={'input_tokens': 444, 'output_tokens': 20, 'total_tokens': 464})]}}
----


In [28]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I apologize, but I don't have access to personal information about individual users, including names. As an AI assistant, I don't have any prior knowledge about you or your personal details. Is there something else I can help you with?", response_metadata={'id': 'msg_01NQMD87MEBeAMdyeqeDskzB', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 402, 'output_tokens': 52}}, id='run-1db9a1f8-05ab-4ff9-8104-0eababaa89cd-0', usage_metadata={'input_tokens': 402, 'output_tokens': 52, 'total_tokens': 454})]}}
----
